In [ ]:
from util.string_helper import word_tokenize, clear_special_characters
import pandas as pd
import numpy as np
import nltk
import json
from util.string_helper import clear_text


In [ ]:
nltk.download('punkt')
droplist = json.load(open("data/constants/header_droplist.json", "r"))


In [ ]:
def preprocessing(text):
    text = str(text)
    text = word_tokenize(text.lower())
    text = clear_special_characters(text)
    text = [word for word in text if len(word) > 1]
    return " ".join(text)


In [ ]:
# Header'ı veya içeriği boş olan ve tekrarlanan sectionların temizlenmesi
df = pd.read_csv("data/acl/sections/sections.csv")
print(df.shape)
df = df.dropna(subset=["header", "content"])
df.replace(np.nan, '', inplace=True)
df.drop_duplicates(subset=["header", "content"], inplace=True)
print(df.shape)


In [ ]:
# Preprocessing fonksiyonu ile temizlenen başlıklardan sonra başlığı olmayan sectionların temizlenmesi
df["header_concat"] = df['parent_header'].astype(str) + " " + df["header"]
df[["header_clean"]] = df[["header_concat"]].applymap(preprocessing)
clean_df = df[(df["header_clean"] != "")]
clean_df.shape


In [ ]:
# Belirlenen keywordleri içeren başlıklara sahip sectionların temizlenmesi

dropped_df = clean_df[~clean_df["header_clean"].str.lower(
).str.contains("|".join(droplist))]
dropped_df.to_csv("data/acl/sections/sections_header_clean.csv", index=False)
dropped_df.shape


In [ ]:
drop_df = clean_df[clean_df["header_clean"].str.lower(
).str.contains("|".join(droplist))]
drop_df.shape


In [ ]:
all_header_freqs = clean_df.groupby("header_clean").count(
).sort_values(by="header", ascending=False)["header"]
dropped_header_freqs = dropped_df.groupby("header_clean").count(
).sort_values(by="header", ascending=False)["header"]
drop_header_freqs = drop_df.groupby("header_clean").count(
).sort_values(by="header", ascending=False)["header"]


In [ ]:
all_header_freqs.to_csv("data/acl/sections/analysis/header_freqs.csv")
dropped_header_freqs.to_csv(
    "data/acl/sections/analysis/dropped_header_freqs.csv")
drop_header_freqs.to_csv("data/acl/sections/analysis/drop_header_freqs.csv")


In [ ]:
cleaned_df = dropped_df.copy()
cleaned_df.drop(columns=["header_concat", "header_clean"], inplace=True)
cleaned_df.header = cleaned_df.header.apply(clear_text)
cleaned_df.parent_header.fillna("", inplace=True)
cleaned_df.parent_header = cleaned_df.parent_header.apply(clear_text)
cleaned_df.content = cleaned_df.content.apply(clear_text)
cleaned_df.dropna(subset=["content"], inplace=True)
cleaned_df.drop(cleaned_df[cleaned_df["header"] == ''].index, inplace=True)
cleaned_df.to_csv('data/acl/sections/sections_clean.csv')
